In [ ]:
import pandas as pd

burns_df = pd.read_csv('burns.csv', low_memory=False)
rugpull_labels_df = pd.read_csv('rugpulls_with_token_info.csv', low_memory=False)

# Ensure that 'id' is the correct identifier for pools
print(burns_df.head())

# Convert 'date' to a datetime object
burns_df['timestamp'] = pd.to_datetime(burns_df['timestamp'], unit='s')


In [ ]:
import pandas as pd

# Initialize a list to store the feature rows
feature_rows = []
# Initialize an empty DataFrame for burn features
burn_features = pd.DataFrame()

import numpy as np

count = 0
# Initialize a list to store the feature rows
feature_rows = []
# Initialize a DataFrame to store the features
pool_features = pd.DataFrame()

for index, pool_id in enumerate(rugpull_labels_df['pool_id']):
    if index % 200 == 0:
        print(f"Processing pool {index}: Pool ID {pool_id}")
        print(f"Number of pools skipped: {count}")
        # if index > 0: break

    rugpull_date = rugpull_labels_df[rugpull_labels_df['pool_id'] == pool_id]['date'].iloc[0] # Get the rugpull date for this pool

    token0_symbol = rugpull_labels_df[rugpull_labels_df['pool_id'] == pool_id]['token0.symbol'].iloc[0]
    token1_symbol = rugpull_labels_df[rugpull_labels_df['pool_id'] == pool_id]['token1.symbol'].iloc[0]

    # print(rugpull_date)
    burn_data = burns_df[burns_df['pool.id'] == pool_id]

    # if (len(burn_data) < 3):
    #     # print(f"Skipping pool {pool_id} because there is no data")
    #     count += 1
    #     continue

    # If rugpull_date is NaT, use all available data; else filter data before rugpull
    pre_rugpull_data = burn_data if pd.isna(rugpull_date) else burn_data[burn_data['timestamp'] < rugpull_date]
    
    if pre_rugpull_data.empty:
        print(f"Skipping pool {pool_id} because there is no data before the rugpull date")
        continue

    # # Calculate the distribution of burned tokens for the top 10 addresses
    # grouped_owners = pre_rugpull_data.groupby('origin')['amount'].sum()
    # print(grouped_owners[0])
    # top_owners_amount = grouped_owners.nlargest(10).sum()
    # total_burned_amount = grouped_owners.sum()
    # distribution_top_10 = top_owners_amount / total_burned_amount if total_burned_amount != 0 else 0

    # total_burned_amount = pre_rugpull_data['amount'].sum()
    # distribution_top_10 = top_owners_amount / total_burned_amount if total_burned_amount != 0 else 0
    
    pre_rugpull_data = pre_rugpull_data.select_dtypes(include=[np.number])
    
    # Identify if token0 or token1 is WETH and rename columns
    if token0_symbol == 'WETH':
        weth_token = 'amount0'
        otherToken = 'amount1'
    elif token1_symbol == 'WETH':
        weth_token = 'amount1'
        otherToken = 'amount0'
    else:
        print(f"Skipping pool {pool_id} because neither token is WETH")
        continue

    # Rename columns
    pre_rugpull_data.columns = [col.lower().replace(weth_token, 'wethToken').replace(otherToken, 'otherToken') for col in pre_rugpull_data.columns]

    # # Debugging: Print new column names
    # print("New Column Names:", pre_rugpull_data.columns)
    
    # Calculate mean and variance for relevant columns
    means = pre_rugpull_data.mean().fillna(0)
    variances = pre_rugpull_data.var().fillna(0)  # Replacing NaN with 0

    feature_row = {}

    feature_row = {'pool_id': pool_id}
    for mean in means.index:
        feature_row[f'mean_{mean}'] = means[mean]
    for variance in variances.index:
        feature_row[f'variance_{variance}'] = variances[variance]
        
    feature_rows.append(feature_row)

# Convert the list of dictionaries to a DataFrame
pool_features = pd.DataFrame(feature_rows)

print(pool_features.head())


In [ ]:
pool_data_with_labels = pd.read_csv('merged_pool_data.csv', low_memory=False)

# Merge the DataFrames on the 'pool_id' column
merged_data = pool_data_with_labels.merge(pool_features, on='pool_id', how='left')

# Save the merged DataFrame to a new CSV file, if needed
merged_data.to_csv('merged_pool_data2.csv', index=False)

print("Merged data saved to merged_pool_data2.csv")